In [1]:
# !pip install IPython --user
# !pip install matplotlib --user
# !pip install ipywidgets --user

In [2]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

<h1 align='center'>Introducting Interactivity into Jupyter Notebooks</h1>

<h4 align='center'>Laura Gutierrez Funderburk$\mid$ SciProg $\mid$ Simon Fraser University</h4>

<h2 align='center'>Abstract</h2>

In this workshop we will explore the use of Jupyter Widgets and how they allow developers to add an interactive element to Jupyter notebooks.

<h2 align='center'>Introduction to Jupyter Widgets</h2>

Jupyter Widgets are eventful python objects that have a representation in the browser, often as a control like a slider, textbox, buttons, among others. 

Widgets can be used to incorporate interactive GUIs within a notebook. Widgets can also be used to "sincronize stateful and stateless information between Python and JavaScript" $^{[1]}$.

In my work as a developer, I have found widgets useful when sharing technical information with audiences who do not have a technical background, for example, when building learning material for K12 students.

<h2 align='center'>Example I: Rolling a Die</h2>

Let us run a simple probability experiment. Suppose we want to roll a die multiple times, and we wish to track and plot the number of times we obtain each face. We can use basic python coding along with Matplotlib to plot what we wish. 

We can also use Jupyter widgets to create a nice interface that allows us to perform several probability experiments and observe the distribution. 

Using the slider below, change the number of trials and press the Run Interact button multiple times to run the experiment. 

In [3]:
#Code written by Laura Gutierrez Funderburk

# Import libraries
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import random
from ipywidgets import widgets, interact_manual,interact

# This function takes as input an integer number_turns indicating the number of times we wish to repeat 
#   experiment
# This function plots two figures: the left hand side figure is a scatter plot where each point is determined
#  by the pair (n,y) where n is the nth trial and y is the outcome obtained after rolling a die
#  The right hand side is a bar plot that displays the frequency of each face after all trials.
def plot_random(number_turns):
    
    # Initialize figure
    fig = plt.figure(figsize=(15,10)) 

    # Initialize left hand side figure
    ax = fig.add_subplot(221)
    
    # Set x and y limits
    ax.set_xlim([0,number_turns + 1])
    ax.set_ylim([0,7])
    
    # Draw grid
    ax.grid(True)
    
    # Compute die outcome as a random choice between integers 1 through 6, repeat for number_turns times
    die_outcome = [random.choice([1,2,3,4,5,6]) for i in range(number_turns)]
    
    # Compute y values 
    y = [i for i in range(1,number_turns+1)]

    # Plot outcome and label plot
    ax.scatter(y,die_outcome,color='#000000',label="Experiment Outcome")
    ax.set_ylabel("Outcome",fontsize=15)
    ax.set_xlabel("Number of trials",fontsize=15)
    ax.set_title("Rolling a Die",fontsize=25)
    
    # Initialize right hand side figure
    ax3 = fig.add_subplot(222)
    
    # Compute frequency of each outcome
    freq = [die_outcome.count(i) for i in range(1,7)]
    
    # Draw grid
    ax3.grid(True)
    
    # Plot outcome and label plot
    ax3.bar([1,2,3,4,5,6],freq)
    ax3.set_ylabel("Frequency",fontsize=15)
    ax3.set_xlabel("Face",fontsize=15)
    ax3.set_title("Distribution",fontsize=25)
    
    # Show plot
    plt.show()
    
# Initalize and configure widget: connect function and manual widget
style = {'description_width': 'initial'}
interact_manual(plot_random,number_turns=widgets.IntSlider(
            value=2,
            min=1,
            max=1000,
            step=1,
            description='Number of trials',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='d',
            style =style
));

interactive(children=(IntSlider(value=2, continuous_update=False, description='Number of trials', max=1000, min=1, style=SliderStyle(description_width='initial')), Button(description='Run Interact', style=ButtonStyle()), Output()), _dom_classes=('widget-interact',))

<h2 align='center'>Example II: Modelling a Disease Outbreak</h2>

Widgets can also be useful when sharing technical information among scientists. 

For instance, suppose we wish to simulate a disease outbreak using a discrete version of the S.I.R. model. Suppose we are given the following data set:

| Date  |Day Number |Susceptible | Infected |
|-------||-------------|----------|
|June 19 1665|0|254|7|
|July 3 1665|14|235|14|
|July 19 1665|28|201|22|
|Aug 3 1665|42|153|29|
|Aug 19 1665|56|121| 21|
|Sept 3 1665|70|108|8|
|Sept 19 1665|84|121|21|
|Oct 3 1665| 98|NA | NA|
|Oct 19 1665|112| 83 | 0|

We can code the discrete version of the SIR model. Suppose we wish to share our implementation of this model with a colleague so that they can perform analysis of the outbreak. 

What if the colleague we wish to share information with does not have a mathematical nor coding background? 

This is where the power of Jupyter Widgets comes in handy. We can provide a visual interface for them to interact with the model. 

Using the sliders and the boxes, change the values of initial conditions. Press the Run Interact button once you are ready and simulate an outbreak. 

In [4]:
# Code written and revised by Laura Gutierrez Funderburk and Dr. Cedric Chauve

import matplotlib.pyplot as plt
import numpy as np
from math import ceil

# This function takes as input a vector y holding all initial values,
#    n the number of time points
#    beta: beta parameter of the SIR 
#    gamma: gamma parameter of the SIR
#    S1,I1,R1 = initial values

def discrete_SIR(S1,I1,R1,n,beta,gamma):
    # Empy arrays for each class
    S = []
    I = []
    R = []
    N = S1+I1+R1
    
    # Append initial values
    S.append(S1)
    I.append(I1)
    R.append(R1)
    
    # apply SIR model: iterate over the total number of days - 1
    for i in range(n-1):
        S_next = S[i] - (beta/N)*((S[i]*I[i]))
        S.append(S_next)
        
        I_next = I[i] + (beta/N)*((S[i]*I[i])) - gamma*I[i]
        I.append(I_next)
        
        R_next = R[i] + gamma * I[i]
        R.append(R_next)
    
    # return arrays S,I,R whose entries are various values for susceptible, infected, removed 
    return((S,I,R))


# This function takes as input initial values of susceptible, infected and removed, number of days, beta and gamma
# it plots the SIR model with the above conditions
def plot_SIR(S1,I1,R1,n,beta,gamma):
    
    # Initialize figure
    fig = plt.figure(facecolor='w',figsize=(17,5))
    ax  = fig.add_subplot(111,facecolor = '#ffffff')
    
    # Compute SIR values for our initial data and parameters
    (S_f,I_f,R_f) = discrete_SIR(S1,I1,R1,n,beta,gamma)    
    
    # Set x axis
    x = [i for i in range(n)]
   
    # Scatter plot of evolution of susceptible over the course of x days
    plt.scatter(x,S_f,c= 'b',label='Susceptible')
    
    # Scatter plot of evolution of infected over the course of x days
    plt.scatter(x,I_f,c='r',label='Infected')
    
    # Scatter plot of evolution of removed over the course of x days
    plt.scatter(x,R_f,c='g',label='Removed')

    # Make the plot pretty
    plt.xlabel('Time (days)')
    plt.ylabel('Number of individuals')
    plt.title('Simulation of a disease outbreak using the SIR model')
    legend = ax.legend()
    plt.show()
    
    # Print messages to aid student understand and interpret what is happening in the plot
    print("SIMULATION DATA\n")
    print("Beta: " + str(beta))
    print("Gamma: " + str(gamma))
    print("\n")
    print("Initial Conditions:")
    print("Total number of Susceptible: "  + str(ceil(S_f[0])))
    print("Total number of Infected: "  + str(ceil(I_f[0])))
    print("Total number of Removed: "  + str(ceil(R_f[0])))
    print("\n")
    print("After " + str(n) + " days:")
    print("Total number of Susceptible: "  + str(ceil(S_f[n-1])))
    print("Total number of Infected: "  + str(ceil(I_f[n-1])) )
    print("Total number of Removed: "  + str(ceil(R_f[n-1])))
# Tweaking initial Values
from ipywidgets import widgets, interact, interact_manual

# Set function above so that the user can set all parameters and manually start simulation
s = {'description_width': 'initial'}
interact_manual(plot_SIR,
        S1  =widgets.IntSlider(value = 254,
                               min = 200,
                               max = 1000,
                               step = 1,
                               style=s,
                               description="Susceptible Initial",
                               disabled=False,
                               orientation='horizontal',
                               readout=True),
        I1 = widgets.IntSlider(value = 7,
                               min = 0,
                               max = 500,
                               step = 1,
                               style=s,
                               description="Infected Initial",
                               disabled=False,
                               orientation='horizontal',
                               readout=True),
         R1 = widgets.IntSlider(value = 0,
                               min = 0,
                               max = 500,
                               step = 1,
                               style=s,
                               description="Removed Initial",
                               disabled=False,
                               orientation='horizontal',
                               readout=True),
         n = widgets.IntSlider(value = 112,
                               min = 0,
                               max = 500,
                               step = 1,
                               style=s,
                               description="Time (days)",
                               disabled=False,
                               orientation='horizontal',
                               readout=True),
         beta = widgets.FloatText(
    value=1.50,
    description=r'$ \beta$ parameter',
    disabled=False,
    style = s,
    step=0.01
),
        gamma = widgets.FloatText(
    value=1.50,
    description= r'$ \gamma$ parameter',
    disabled=False,
    style = s,
    step=0.01
)
        );


interactive(children=(IntSlider(value=254, description='Susceptible Initial', max=1000, min=200, style=SliderStyle(description_width='initial')), IntSlider(value=7, description='Infected Initial', max=500, style=SliderStyle(description_width='initial')), IntSlider(value=0, description='Removed Initial', max=500, style=SliderStyle(description_width='initial')), IntSlider(value=112, description='Time (days)', max=500, style=SliderStyle(description_width='initial')), FloatText(value=1.5, description='$ \\beta$ parameter', step=0.01, style=DescriptionStyle(description_width='initial')), FloatText(value=1.5, description='$ \\gamma$ parameter', step=0.01, style=DescriptionStyle(description_width='initial')), Button(description='Run Interact', style=ButtonStyle()), Output()), _dom_classes=('widget-interact',))

<h2 align='center'>Summary</h2>

In this notebooks we learned how Jupyter Widgets can help up introduce interactivity into our notebooks and explored two examples in different scenarios. 

In the next notebook, you will have an opportunity to learn about different Jupyter Widgets and to implement your own widget into a simple Python function.

<h2 align='center'>References</h2>

$[1]$ https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20Basics.html